In [203]:
token="pat-eu1-52e0d25c-7a77-4128-bf28-4787e96528bf"


# Bibliothèques
from hubspot.crm.contacts import SimplePublicObjectInput
from hubspot.crm.contacts.exceptions import ApiException
from hubspot import HubSpot


In [204]:
# Connection à l'API
api_client = HubSpot(access_token=token)

# Valeur totale des deals avec et sans pondération

### Récupération des infos de l'entreprise

In [205]:
id=7496989169
# Récupération de toute les propriété d'une entreprise
# api_client.crm.properties.core_api.get_all(object_type="companies")

# properties=["name","city","country","industry","hs_analytics_num_visits","hs_annual_revenue_currency_code","total_money_raised","total_revenue","valeur_totale_des_deals","valeur_totale_des_deals_ponderee","owneremail","numberofemployees","hs_total_deal_value","total_money_raised","hs_predictivecontactscore_v2","hs_target_account_probability","hs_predictivecontactscore_v2_next_max_max_d4e58c1e","num_associated_contacts","num_conversion_events","num_conversion_events_cardinality_sum_d095f14b"]
properties_needed=["hs_predictivecontactscore_v2","valeur_totale_des_deals","valeur_totale_des_deals_ponderee","nombre_de_points_de_contacts_en_poste","hubspot_owner_id"]
company=api_client.crm.companies.basic_api.get_by_id(id,properties=properties_needed)
company_data=company.to_dict()
company_data["properties"]

{'createdate': '2023-04-28T09:53:40.624Z',
 'hs_lastmodifieddate': '2023-04-30T13:19:47.041Z',
 'hs_object_id': '7496989169',
 'hs_predictivecontactscore_v2': '3.6',
 'hubspot_owner_id': '919776484',
 'nombre_de_points_de_contacts_en_poste': '8',
 'valeur_totale_des_deals': '405498.0',
 'valeur_totale_des_deals_ponderee': '197168.2'}

# Récupération des infos des deals

In [206]:
# Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
deals_id=[]
deals=api_client.crm.deals.get_all()
for deal in deals:
    deals_id.append(deal.to_dict()["id"])
deals_id

['7280660186', '7280697021', '7280697041']

In [207]:
# deal1=7280660186
# deal2=7280697021
# deal3=7280697041
# Récupération de toute les propriété d'un deal
# api_client.crm.properties.core_api.get_all(object_type="deals")

#properties=["amount","hs_likelihood_to_close","hs_predicted_amount","hs_priority","hs_closed_amount","hs_deal_stage_probability","hs_deal_stage_probability_shadow","hs_forecast_amount","hs_forecast_probability","hs_is_closed","hs_is_closed_won","hs_projected_amount","num_associated_contacts"]
properties_needed=["amount","hs_deal_stage_probability"]
for id in deals_id :
    deal=api_client.crm.deals.basic_api.get_by_id(id,properties=properties_needed)
    deal_data=deal.to_dict()
    print(f"Deal: {id} ")
    print(deal_data["properties"])

Deal: 7280660186 
{'amount': '25400', 'createdate': '2023-04-28T09:55:04.007Z', 'hs_deal_stage_probability': '0.200000000000000011102230246251565404236316680908203125', 'hs_lastmodifieddate': '2023-04-30T01:31:20.094Z', 'hs_object_id': '7280660186'}
Deal: 7280697021 
{'amount': '300000', 'createdate': '2023-04-28T09:56:12.025Z', 'hs_deal_stage_probability': '0.40000000000000002220446049250313080847263336181640625', 'hs_lastmodifieddate': '2023-04-29T12:14:08.235Z', 'hs_object_id': '7280697021'}
Deal: 7280697041 
{'amount': '80098', 'createdate': '2023-04-28T09:57:05.960Z', 'hs_deal_stage_probability': '0.90000000000000002220446049250313080847263336181640625', 'hs_lastmodifieddate': '2023-04-29T12:13:43.668Z', 'hs_object_id': '7280697041'}


In [208]:
# Récupération du montant total des deals sans pondération
#deals_id=[deal1,deal2,deal3]
montant_deals=0
for id in deals_id:
    deal=api_client.crm.deals.basic_api.get_by_id(id)
    deal_data=deal.to_dict()
    montant_deals+=float(deal_data["properties"]["amount"])
print(montant_deals)


405498.0


In [248]:
# Calcul du montant total des deals avec pondération

def total_deal_value_weighted(properties):

    # Recupération des ids des deals (si on considère que tous les deals sont associés à une seule entreprise)
    deals_id=[]
    deals=api_client.crm.deals.get_all()
    for deal in deals:
        deals_id.append(deal.to_dict()["id"])
    
    # Calcul du montant total des deals avec pondération
    montant_deals_pond=0
    for id in deals_id:
        deal=api_client.crm.deals.basic_api.get_by_id(id,properties=properties_needed)
        deal_data=deal.to_dict()
        montant_deals_pond+=float(deal_data["properties"]["amount"]) * float(deal_data["properties"]["hs_deal_stage_probability"])
    return montant_deals_pond

total_deal_value_weighted()


KeyError: 'amount'

### Update du montant d'un deal


In [210]:
# deal_id=""
# new_amount= 0
# update_montant_deal = SimplePublicObjectInput(
#         properties={"amount":new_amount})
# api_client.crm.companies.basic_api.update(simple_public_object_input=update_montant_deal,company_id=deal_id)

# Contacts

In [247]:
# Calcul du nombre de contacts dont En poste = Oui

def count_en_poste(etat="Oui"):

    # Récupération des id des contacts
    contacts_id=[]
    contacts=api_client.crm.contacts.get_all()
    for contact in contacts:
        contacts_id.append(contact.to_dict()["id"])

    # Calcul du nombre de contact selon l'état de l'attribut en poste
    cpt_en_poste=0
    for id in contacts_id:
        contact_data=api_client.crm.contacts.basic_api.get_by_id(id,properties=["en_poste"]).to_dict()
        if contact_data["properties"]["en_poste"] == etat:
            cpt_en_poste+=1
    return cpt_en_poste
count_en_poste()


8

In [245]:
# Propriétaire actuel

def update_owner():
    owner_id=api_client.crm.companies.basic_api.get_by_id(id,properties=["hubspot_owner_id"]).to_dict()
    owner_id=owner_id["properties"]["hubspot_owner_id"]
    owner_data=api_client.crm.owners.owners_api.get_by_id(owner_id).to_dict()
    return [["first_name"],["last_name"]]

# Mise à jour des infos de l'entreprise

In [240]:
# Mise à jour des infos de l'entreprise
comp_id="7496989169"
properties={
                "valeur_totale_des_deals":montant_deals ,
                "valeur_totale_des_deals_ponderee":montant_deals_pond,
                "nombre_de_points_de_contacts_en_poste": count_en_poste(),
                "nom_du_dirigeant":update_owner()[1],
                "prenom_du_dirigeant":update_owner()[2]}
def update_company(comp_id,properties):
        try:
                update_montant_tot_deals = SimplePublicObjectInput(properties=properties)
                api_client.crm.companies.basic_api.update(simple_public_object_input=update_montant_tot_deals,company_id=comp_id)
                return print("company updated")
        except ApiException as update:
                return print("Exception when updating company: %s\n" % update)
update_company(comp_id,properties)
        

company updated


In [239]:
# Récupération des infos de l'entreprise

comp_id=7496989169
properties=["name","city","country","industry","nom_du_dirigeant","prenom_du_dirigeant","numberofemployees","valeur_totale_des_deals","valeur_totale_des_deals_ponderee","num_associated_deals","nombre_de_points_de_contacts_en_poste","nombre_de_points_de_contacts_en_poste"]
def comp_info(comp_id,properties):
    company_data=api_client.crm.companies.basic_api.get_by_id(id,properties=properties).to_dict()
    return company_data["properties"]
comp_info(comp_id,properties)

{'city': 'Grenoble',
 'country': 'France',
 'createdate': '2023-04-28T09:53:40.624Z',
 'hs_lastmodifieddate': '2023-04-30T14:20:50.150Z',
 'hs_object_id': '7496989169',
 'industry': 'INFORMATION_TECHNOLOGY_AND_SERVICES',
 'name': 'company',
 'nom_du_dirigeant': 'company',
 'nombre_de_points_de_contacts_en_poste': '8',
 'num_associated_deals': '3',
 'numberofemployees': '20',
 'prenom_du_dirigeant': 'owner',
 'valeur_totale_des_deals': '405498.0',
 'valeur_totale_des_deals_ponderee': '197168.2'}